### compute mean power across 3s window

In [1]:
import numpy as np
from os import path, listdir, makedirs
from itertools import product

from brainpipe.system import study
from utils import odor_groups_3wgth

st = study('Olfacto')
PATH = path.join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all')
pow_file = path.join(PATH, '{}_odor_{}_{}_6freqs.npz')
exp = 'E'

for su in odor_groups_3wgth:
    for cond in odor_groups_3wgth[su]:
        mat = np.load(pow_file.format(su,cond,exp),allow_pickle=True)
        mean_pow = np.mean(mat['xpow'],axis=2)
        new_data = {}
        for file in mat.files:
            new_data[file] = mat[file]
        new_data['xpow'] = mean_pow
        np.savez(pow_file.format(su,cond,exp).replace('.npz','_mean.npz'), **new_data)


-> Olfacto loaded


### compute Linear Regression analysis using mean power

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from mne.stats import fdr_correction, bonferroni_correction
from utils import rename_elecs, odors_su_score

"""
Correlate TPSim with behavioral variables // Memory group
BY ELECTRODE and Plot summary (for all included electrodes)
"""

exps = ['Enc']
freqs,f = ['theta'], 1
meth, conds, stat = 'btw', ['low','mid','high'], 'LinReg'
subjects = ['CHAF','VACJ','PIRJ','SEMC','FERJ','LEFC']
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf',
            'OFC_olf','SFG']
st = study('Olfacto')

for exp in exps:
    ###############################################################################
    path_pow = path.join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond_3groups_v=1_elecs=all/')
    pow_form = path.join(path_pow, '{}_odor_{}_'+exp[0]+'_6freqs_mean.npz')
    df_path = path.join(st.path, 'feature/TPSim_3groups_{}/LinReg_stats_pow_mean_{}_v=1_elecs=all/')
    df_name = path.join(df_path, '{}_ols_'+meth+'_{}_{}_{}.csv') #su, conds0, conds1, freq
    ###############################################################################
    
    dict_perf = {'low':1, 'mid':2, 'high':3}

    for freq in freqs:
        if not path.exists(df_path.format(exp,freq)):
            makedirs(df_path.format(exp,freq))
        subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
        channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
        pow_scores, T_vals_c, p_vals_c = np.array([]), np.array([]), np.array([])
        p_fdr_c, p_bf_c = np.array([]), np.array([])
        
        for su in subjects:
            #load all elec info,rename and select electrodes id
            mat = np.load(pow_form.format(su,conds[0]),allow_pickle=True)
            labels, channels = mat['labels'], mat['channels']
            print(labels.shape, mat['xpow'].shape)
            x, y, z = mat['xyz'][:,0], mat['xyz'][:,1], mat['xyz'][:,2]
            labels_new = rename_elecs(labels,x,y,z)
            idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
            labels, channels = labels_new[idx_sel], channels[idx_sel]
            x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]
            
            if stat == 'LinReg':
                tps_su = np.zeros((len(idx_sel),len(conds)))
                all_tps, all_scores = [], []
                for c,cond in enumerate(conds):
                    mat = np.load(pow_form.format(su,cond))
                    pow_ = mat['xpow'][f,idx_sel]
                    nelecs, ntrials = pow_.shape
                    score_ = np.array([dict_perf[cond]]*ntrials)
                    tps_su[:,c] += np.mean(pow_,axis=1)
                    #fill tps and score vectors
                    all_tps.append(pow_)
                    all_scores.append(score_)

                #compute stats Ttests-unpaired
                all_tps = np.concatenate(all_tps,axis=1)
                all_scores = np.concatenate(all_scores,axis=0)
                T, unc_p = [], []
                for elec in range(nelecs):
                    #Tval,pval = stats.kendalltau(all_tps[elec],all_scores)
                    Y, X = np.array(all_tps[elec]), sm.add_constant(np.array(all_scores))
                    model_ols = sm.OLS(Y,X).fit()
                    Tval, pval = np.round(model_ols.tvalues[1],3),model_ols.pvalues[1]
                    T.append(Tval), unc_p.append(pval)
            if stat == 'Ttest':
                tps0 = np.load(tps_form.format(su,conds[0],freq))['tps'][idx_sel]
                nelecs = len(idx_sel)
                tps1 = np.load(tps_form.format(su,conds[1],freq))['tps'][idx_sel]
                mean0, mean1 = np.mean(tps0,axis=1)[:,np.newaxis], np.mean(tps1,axis=1)[:,np.newaxis]
                tps_su = np.concatenate((mean0,mean1),axis=1)
                #compute stats Ttests-unpaired
                tps0, tps1 = tps0.swapaxes(0,1), tps1.swapaxes(0,1) #ntrials x nelecs
                T, unc_p = ttest_ind(tps0, tps1, equal_var=False)
                print(T.shape, unc_p.shape)
            _, p_fdr = fdr_correction(unc_p)
            _, p_bf = bonferroni_correction(unc_p)

            #fill all df data
            subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
            elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
            labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
            channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
            pow_scores = np.concatenate((pow_scores,tps_su),axis=0) if np.size(pow_scores) else tps_su
            x_c = np.hstack((x_c,x)) if np.size(x_c) else x
            y_c = np.hstack((y_c,y)) if np.size(y_c) else y
            z_c = np.hstack((z_c,z)) if np.size(z_c) else z
            T_vals_c = np.hstack((T_vals_c,T)) if np.size(T_vals_c) else T
            p_vals_c = np.hstack((p_vals_c,unc_p)) if np.size(p_vals_c) else unc_p
            p_fdr_c = np.hstack((p_fdr_c,p_fdr)) if np.size(p_fdr_c) else p_fdr
            p_bf_c = np.hstack((p_bf_c,p_bf)) if np.size(p_bf_c) else p_bf

        data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                    channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                    z_c[:,np.newaxis],elecs_c[:,np.newaxis],pow_scores,T_vals_c[:,np.newaxis],
                    p_vals_c[:,np.newaxis],p_fdr_c[:,np.newaxis],p_bf_c[:,np.newaxis]),
                    axis=1)
        df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                                'elecs_num', 'pow_'+conds[0], 'pow_'+conds[1],'pow_'+conds[2],
                                        'Tvals', 'unc_p','fdr_p', 'bonf_p'])
        print(df.shape)
        df.to_csv(df_name.format(exp,freq,'All_subjects',freq,'mean_pow',stat),index=False)   

-> Olfacto loaded
(61,) (6, 61, 5)
(39,) (6, 39, 9)
(24,) (6, 24, 14)
(53,) (6, 53, 21)
(32,) (6, 32, 16)
(27,) (6, 27, 23)
(236, 14)


### summarize

In [4]:
from collections import Counter

st = study('Olfacto')
exp, stat = 'Enc', 'LinReg'
meth = 'btw'
freq = 'theta'
thrs = [0.05]
corrections = ['fdr_p']

##################################################################################
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/LinReg_stats_pow_mean_{}_v=1_elecs=all/')
df_name = path.join(path_pow, 'All_subjects_ols_btw_theta_mean_pow_LinReg.csv')
df_stat_save = path.join(df_name, 'Bilan_{}_OLS_'+meth+'_{}_{}_{}_{}_{}_{}.csv')
df_stat_all = path.join(df_name, 'Bilan_{}_OLS_'+meth+'_{}_{}_{}_{}.csv')
##################################################################################

df = pd.read_csv(df_name.format(freq))
print('Initial df shape', df.shape, meth, exp, stat)

for th, corr in product(thrs,corrections):
    df_sel = df.loc[df[corr]<th]
    df_sel['sign'] = ['separation' if t > 0 else 'completion' for t in df_sel['Tvals']]
    print('\n',meth,'stats at p < ',th, 'correction : ',corr, df_sel.shape)
    print(df_sel)
    
    rois = np.unique(df_sel['labels'])
    for roi in rois:
        df_roi = df_sel.loc[df_sel['labels']==roi]
        df_inc = df_roi.loc[df_roi['sign']=='completion'].groupby(['subjects']).count()
        df_dec = df_roi.loc[df_roi['sign']=='separation'].groupby(['subjects']).count()

        if (df_inc.shape[0] >= 3):
            print(roi, 'NB of subjects with completion',df_inc.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='completion']
            print('#electrodes in total >>> ',df_plot.shape[0])
            df_plot.to_csv(df_stat_save.format(freq,'All_subjects','theta','mean_pow',
                                               'compl',roi,corr+str(th),stat))

        if (df_dec.shape[0] >= 3):
            print(roi, 'NB of subjects with separation',df_dec.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='separation']
            print('#electrodes in total >>> ',df_plot.shape[0])
            df_plot.to_csv(df_stat_save.format(freq,'All_subjects','theta','mean_pow',
                                               'sep',roi,corr+str(th),stat))
            #print(df_plot)

-> Olfacto loaded
Initial df shape (236, 14) btw Enc LinReg

 btw stats at p <  0.05 correction :  fdr_p (7, 15)
    subjects   labels   channels      x      y      z  elecs_num   pow_low  \
35      CHAF      SFG  f'14-f'13 -21.00  14.75  65.80         35  2.400411   
155     SEMC      IFG      u9-u8  37.15  43.40  -8.60         31  1.394665   
166     SEMC      SFG      y4-y3  20.00  52.55  27.80         42  0.684818   
186     FERJ      aHC    b'4-b'3 -30.45 -17.20 -13.40          9  3.030832   
187     FERJ      aHC    b'5-b'4 -34.40 -17.25 -13.65         10  3.223706   
222     LEFC      MFG      k9-k8  36.40  46.35  12.25         13  0.581131   
226     LEFC  OFC_olf      o3-o2  13.25  36.75 -13.35         17  0.571784   

      pow_mid  pow_high  Tvals     unc_p     fdr_p    bonf_p        sign  
35   0.255572 -1.220136 -4.908  0.000098  0.005963  0.005963  completion  
155  0.497870 -0.189076 -3.750  0.000589  0.031208  0.031208  completion  
166  0.382506  4.278172  3.378  0.001